In [ ]:
import numpy as np
from collections import deque
import pandas as pd
import math
import time
import random
from os import chdir, getcwd, listdir
from os.path import isfile

In [ ]:
def Euc_2D(lista):
  n = len(lista)
  matriz = []
  for i in range(n):
    aux = []
    for j in range(n):
      aux.append(np.sqrt((lista[i][0]-lista[j][0])**2 + (lista[i][1] - lista[j][1])**2))
    matriz.append(aux)
  return matriz

In [ ]:
def Euc_2D(lista):
  n = len(lista)
  matriz = []
  for i in range(n):
    aux = []
    for j in range(n):
      if i == j:
        aux.append(99999999999)
      else:
        aux.append(np.sqrt((lista[i][0]-lista[j][0])**2 + (lista[i][1] - lista[j][1])**2))
    matriz.append(aux)
  return matriz

In [ ]:
def Calculate_FO(estrutura_total, dist):
  custo_total = 0
  for i, n in enumerate(custo_caminhao):
    if estrutura_total[i]:
      custo_rota_caminhao = Calculo_da_FO_Caixeiro(estrutura_total[i], dist, n)
      custo_total = custo_total + custo_rota_caminhao
  return custo_total

def Calculo_da_FO_Caixeiro(lista, dist, custo_caminhao):
  anterior = 0
  fo = 0
  for i in range(len(lista)):
    fo = fo + dist[anterior][lista[i]]
    anterior = lista[i]
  ##fo = fo + dist[anterior][0]
  return fo*custo_caminhao

def def_estrutura_de_dados(n, m, quantidade_volume, quantidade_peso, quantidade_tempo, origem):
  estrutura_total = [[] for _ in range(m)]
  fo = []
  volume = []
  peso = []

  for i in range(m):
    fo.append(0)
    volume.append([quantidade_volume[i],0])
    peso.append([quantidade_peso[i],0])
    tempo.append([quantidade_tempo[i],0])

  estrutura_total.append(fo)
  estrutura_total.append(volume)
  estrutura_total.append(peso)
  estrutura_total.append(tempo)
  estrutura_total.append(origem)
  return estrutura_total

# Heurística Construtiva

In [ ]:
'''
    if len(estrutura_total[0]) == 0:
        cost_atual = dist[0][i] + dist[i][0]
        caixeiro = 0
    else:
      fo1 = estrutura_total[m][0]
      cost_atual = fo1 + dist[estrutura_total[0][-1]][i] + dist[i][0] - dist[estrutura_total[0][-1]][0]
      caixeiro = 0

'''

def Construtive_Insertion(estrutura_total, n, m, dist, tradutor_volume, tradutor_massa, custo_caminhao):
  lista = [i for i in range(1, n)]
  random.shuffle(lista)

  for i in lista:
    volume = tradutor_volume[i]
    massa = tradutor_massa[i]
    cost_atual = 100000000
    for j in range(0, m):
      fo1 = estrutura_total[m][j]
      caixeiro = j
      if estrutura_total[m+1][j][1] + volume <= estrutura_total[m+1][j][0] and estrutura_total[m+2][j][1] + massa <= estrutura_total[m+2][j][0]: 
        if len(estrutura_total[j]) == 0:
          cost_aux1 = dist[estrutura_total[j][-1]][j] + dist[j][estrutura_total[j][-1]]
        else:
          cost_aux1 = fo1 + dist[estrutura_total[j][-1]][i] + dist[i][0] - dist[estrutura_total[j][-1]][0]
      if cost_aux1 < cost_atual:
        caixeiro_for_real = caixeiro
        cost_atual = cost_aux1

    estrutura_total[caixeiro_for_real].append(i)
    estrutura_total[m][caixeiro_for_real] = cost_atual
    estrutura_total[m+2][caixeiro_for_real][1] = estrutura_total[m+2][caixeiro_for_real][1] + massa
    estrutura_total[m+1][caixeiro_for_real][1] = estrutura_total[m+1][caixeiro_for_real][1] + volume
  return estrutura_total

# Herística de Busca Local

In [ ]:
from IPython.core.display import display_latex
def swap_inter(estrutura_total, valor1, valor2, caixeiro1, caixeiro2, m, dist, tradutor_volume, tradutor_massa, custo_caminhao):
  save = estrutura_total
  indice1 = estrutura_total[caixeiro1].index(valor1)
  indice2 = estrutura_total[caixeiro2].index(valor2)
  volume1 = tradutor_volume[valor1]
  volume2 = tradutor_volume[valor2]
  massa1 = tradutor_massa[valor1]
  massa2 = tradutor_massa[valor2]

  estrutura_total[caixeiro1][indice1], estrutura_total[caixeiro2][indice2] = estrutura_total[caixeiro2][indice2], estrutura_total[caixeiro1][indice1]
  estrutura_total[m+1][caixeiro1][1] = estrutura_total[m+1][caixeiro1][1] + volume2 - volume1
  estrutura_total[m+1][caixeiro2][1] = estrutura_total[m+1][caixeiro2][1] + volume1 - volume2
  estrutura_total[m+2][caixeiro1][1] = estrutura_total[m+2][caixeiro1][1] + massa2 - massa1
  estrutura_total[m+2][caixeiro2][1] = estrutura_total[m+2][caixeiro2][1] + massa1 - massa2

  estrutura_total[m][caixeiro1] = Calculo_da_FO_Caixeiro(estrutura_total[caixeiro1], dist, custo_caminhao[caixeiro1])
  estrutura_total[m][caixeiro2] = Calculo_da_FO_Caixeiro(estrutura_total[caixeiro2], dist, custo_caminhao[caixeiro2])

  if Calculate_FO(save, dist) > Calculate_FO(estrutura_total, dist) and estrutura_total[m+2][caixeiro1][1] <= estrutura_total[m+2][caixeiro1][0] and estrutura_total[m+2][caixeiro2][1] <= estrutura_total[m+2][caixeiro2][0] and estrutura_total[m+1][caixeiro1][1] <= estrutura_total[m+1][caixeiro1][0] and estrutura_total[m+1][caixeiro2][1] <= estrutura_total[m+1][caixeiro2][0]:
    return estrutura_total
  else:
    return save

def swap_intra(estrutura_total, valor1, valor2, caixeiro, dist, m, custo_caminhao):
  indice1 = estrutura_total[caixeiro].index(valor1)
  indice2 = estrutura_total[caixeiro].index(valor2)
  save = estrutura_total
  estrutura_total[caixeiro][indice1], estrutura_total[caixeiro][indice2] = estrutura_total[caixeiro][indice2], estrutura_total[caixeiro][indice1]

  ## Falta a parte de FO  
  aux1 = Calculo_da_FO_Caixeiro(estrutura_total[caixeiro], dist, custo_caminhao[caixeiro])
  if aux1 <= estrutura_total[m][caixeiro] and aux1 > 0:
    estrutura_total[m][caixeiro] = aux1
  else:
    estrutura_total = save
  return estrutura_total

def shift_inter(estrutura_total, valor1, valor2, caixeiro1, caixeiro2, dist, m, tradutor_volume, tradutor_massa, custo_caminhao):
  indice1 = estrutura_total[caixeiro1].index(valor1)
  indice2 = estrutura_total[caixeiro2].index(valor2)
  volume1 = tradutor_volume[valor1]
  volume2 = tradutor_volume[valor2]
  massa1 = tradutor_massa[valor1]
  massa2 = tradutor_massa[valor2]
  save = estrutura_total

  save1 = estrutura_total[caixeiro1]
  save2 = estrutura_total[caixeiro2]
  aux1 = estrutura_total[caixeiro1][indice1]
  aux2 = estrutura_total[caixeiro2][indice2]

  estrutura_total[caixeiro1].remove(estrutura_total[caixeiro1][indice1])
  estrutura_total[caixeiro2].remove(estrutura_total[caixeiro2][indice2])
  estrutura_total[caixeiro2].insert(0, aux1)
  estrutura_total[caixeiro1].insert(0, aux2)
  estrutura_total[m+1][caixeiro1][1] = estrutura_total[m+1][caixeiro1][1] + volume2 - volume1
  estrutura_total[m+1][caixeiro2][1] = estrutura_total[m+1][caixeiro2][1] + volume1 - volume2
  estrutura_total[m+2][caixeiro1][1] = estrutura_total[m+2][caixeiro1][1] + massa2 - massa1
  estrutura_total[m+2][caixeiro2][1] = estrutura_total[m+2][caixeiro2][1] + massa1 - massa2

  ## Falta a parte de FO
  aux1 = Calculo_da_FO_Caixeiro(estrutura_total[caixeiro1], dist, custo_caminhao[caixeiro1]) 
  aux2 = Calculo_da_FO_Caixeiro(estrutura_total[caixeiro2], dist, custo_caminhao[caixeiro2]) 

  if (aux1 <= estrutura_total[m][caixeiro1] or aux2 <= estrutura_total[m][caixeiro2]) and estrutura_total[m+2][caixeiro1][1] <= estrutura_total[m+2][caixeiro1][0] and estrutura_total[m+2][caixeiro2][1] <= estrutura_total[m+2][caixeiro2][0] and estrutura_total[m+1][caixeiro1][1] <= estrutura_total[m+1][caixeiro1][0] and estrutura_total[m+1][caixeiro2][1] <= estrutura_total[m+1][caixeiro2][0]:
    estrutura_total[m][caixeiro1] = aux1
    estrutura_total[m][caixeiro2] = aux2
  else:
    estrutura_total = save

  return estrutura_total

def Opt_2(estrutura_total, caixeiro, n, matriz_de_distancias, m, custo_caminhao):
  t = len(estrutura_total[caixeiro])
  i = 0
  j = 0
  Mi_af = 0
  Mj = 0
  foOpt = 0;
  
  for i in range(t):
    j = i+2
    while (j+1)%t != i:
      vi  = estrutura_total[caixeiro][i]
      vi_af = estrutura_total[caixeiro][(i+1)%t]
      vj  = estrutura_total[caixeiro][j%t]
      vj_af = estrutura_total[caixeiro][(j+1)%t]
      foOpt = - matriz_de_distancias[vi][vi_af] - matriz_de_distancias[vj][vj_af] + matriz_de_distancias[vi][vj] + matriz_de_distancias[vi_af][vj_af]
      
      if foOpt < 0:
        Mi_af = (i+1)%t
        Mj  = j%t
        inicio = Mi_af
        fim = Mj

        if inicio > fim:
          tam = t - inicio + fim + 1
        else:
          tam = fim - inicio + 1

        p1 = inicio
        p2 = fim
          
        for k in range(int(tam/2)):
          estrutura_total[caixeiro][p1%t], estrutura_total[caixeiro][p2%t] = estrutura_total[caixeiro][p2%t], estrutura_total[caixeiro][p1%t]
          if p1 == t-1:
            p1 = 0
          else:
            p1 = p1+1
          if (p2 == 0):
            p2 = t-1  
          else:
            p2 = p2-1
          estrutura_total[m][caixeiro] = Calculo_da_FO_Caixeiro(estrutura_total[caixeiro], matriz_de_distancias, custo_caminhao[caixeiro])
      j = j+1
  return estrutura_total

def Node_exchange(estrutura_total, caixeiro, m, dist, custo_caminhao):
  save = estrutura_total.copy()
  n = len(save[caixeiro])
  for i in range(n-1):
    j = i+2;
    while j < n:
      if i != 0 and j != n-1:
        vi  = save[caixeiro][i]
        viP = save[caixeiro][i+1]
        viM = 0
        if (i == 0):
          viM = save[caixeiro][n-1]
        else:
          viM = save[caixeiro][i-1]
                
        vj  = save[caixeiro][j]
        vjM = save[caixeiro][j-1]
        vjP = 0
        if j < n-1:
          vjP = save[caixeiro][j+1]
        else:
          vjP = save[caixeiro][0]

        foOpt = - dist[viM][vi] - dist[vi][viP] - dist[vjM][vj] - dist[vj][vjP] + dist[viM][vj] + dist[vj][viP] + dist[vjM][vi] + dist[vi][vjP]

        if foOpt < 0:
          aux = save[caixeiro][i]
          save[caixeiro][i] = save[caixeiro][j]
          save[caixeiro][j] = aux
          save[m][caixeiro] = Calculo_da_FO_Caixeiro(save[caixeiro], matriz_de_distancias, custo_caminhao[caixeiro])

      j = j + 1

  return save

def Opt_3(estrutura_total, caixeiro, dist, m, custo_caminhao): 
  n = len(estrutura_total[caixeiro])
  save = estrutura_total.copy()
  t = n
  for i in range(1, n):
    for j in range(i+2, n):
      for k in range(j+2, n+1):
        A = save[caixeiro][i-1]
        B = save[caixeiro][i]
        C = save[caixeiro][j-1]
        D = save[caixeiro][j]
        E = save[caixeiro][k-1] 
        F = save[caixeiro][k%t]

        d0 = (dist[A][B] + dist[C][D] + dist[E][F]);
        d3 = (dist[A][D] + dist[E][B] + dist[C][F]);

        if d0 > d3:
          tmp = []
          z = j
          while z <= k-1:
            tmp.append(save[caixeiro][z])
            z = z+1
          z = i
          while z <= j-1:
            tmp.append(save[caixeiro][z])
            z = z+1
          
          pos = 0
          f = i
          while f <= k-1:
            save[caixeiro][f] = tmp[pos]
            pos = pos+1
            f = f+1

          save[m][caixeiro] = Calculo_da_FO_Caixeiro(save[caixeiro], matriz_de_distancias, custo_caminhao[caixeiro])
  return save

In [ ]:
def LS(corrente, dist, m, n, tradutor_volume, tradutor_massa, custo_caminhao):
  escolha = 4 ##random.randrange(1, 5)

  if escolha == 1:
    for k in range(0, m-1):
      caixeiro1 = k
      caixeiro2 = k+1
      for i in range(len(corrente[caixeiro1])):
        for j in range(len(corrente[caixeiro2])):
          valor1 = corrente[caixeiro1][i]
          valor2 = corrente[caixeiro2][j]
          corrente1 = swap_inter(corrente, valor1, valor2, caixeiro1, caixeiro2, m, dist, tradutor_volume, tradutor_massa, custo_caminhao)
    return corrente1

  if escolha == 2:
    for j in range(0, m):
      caixeiro = j
      for i in range(len(corrente[caixeiro])-1):
        valor1 = corrente[caixeiro][i]
        valor2 = corrente[caixeiro][i+1]
        corrente1 = swap_intra(corrente, corrente[caixeiro][corrente[caixeiro].index(valor1)], corrente[caixeiro][corrente[caixeiro].index(valor2)], caixeiro, dist, m, custo_caminhao)
    return corrente1

  if escolha == 3:
    for k in range(0, m-1):
      caixeiro1 = k
      caixeiro2 = k+1
      for i in range(len(corrente[caixeiro1])):
        for j in range(len(corrente[caixeiro2])):
          valor1 = corrente[caixeiro1][i]
          valor2 = corrente[caixeiro2][j]
          corrente1 = shift_inter(corrente, valor1, valor2, caixeiro1, caixeiro2, dist, m, tradutor_volume, tradutor_massa, custo_caminhao)
    return corrente1

  if escolha == 4:
    for k in range(0, m):
      caixeiro = k
      corrente1 = Opt_2(corrente, caixeiro, n, dist, m, custo_caminhao)
    return corrente1

def LS_VND(corrente, dist, m, n, tradutor_volume, tradutor_massa, custo_caminhao):
  escolha = 1
  best = corrente

  while True:
    while escolha == 1:
      save = 0
      for k in range(0, m):
        caixeiro = k
        corrente1 = Opt_2(corrente, caixeiro, n, dist, m, custo_caminhao)
        if Calculate_FO(corrente1, dist) < Calculate_FO(best, dist):
          best = corrente1
          save = 1
      if save == 1:
        escolha = 1
      else:
        escolha = 2

    while escolha == 2:
      save = 0
      for k in range(0, m-1):
        caixeiro1 = k
        caixeiro2 = k+1
        for i in range(len(corrente[caixeiro1])):
          for j in range(len(corrente[caixeiro2])):
            valor1 = corrente[caixeiro1][i]
            valor2 = corrente[caixeiro2][j]
            corrente1 = swap_inter(corrente, valor1, valor2, caixeiro1, caixeiro2, m, dist, tradutor_volume, tradutor_massa, custo_caminhao)
            if Calculate_FO(corrente1, dist) < Calculate_FO(best, dist):
              best = corrente1
              save = 1
      if save == 1:
        escolha = 1
      else:
        escolha = 3

    while escolha == 3:
      save = 0
      for k in range(0, m):
        caixeiro = k
        corrente1 = Node_exchange(corrente, caixeiro, m, dist, custo_caminhao)
        if Calculate_FO(corrente1, dist) < Calculate_FO(best, dist):
          best = corrente1
          save = 1
      if save == 1:
        escolha = 1
      else:
        escolha = 4

    while escolha == 4:
      save = 0
      for k in range(0, m):
        caixeiro = k
        corrente1 = Opt_3(corrente, caixeiro, dist, m, custo_caminhao)
        if Calculate_FO(corrente1, dist) < Calculate_FO(best, dist):
          best = corrente1
          save = 1
      if save == 1:
        escolha = 1
      else:
        escolha = 5

    while escolha == 5:
      save = 0
      for k in range(0, m-1):
        caixeiro1 = k
        caixeiro2 = k+1
        for i in range(len(corrente[caixeiro1])):
          for j in range(len(corrente[caixeiro2])):
            valor1 = corrente[caixeiro1][i]
            valor2 = corrente[caixeiro2][j]
            corrente1 = swap_inter(corrente, valor1, valor2, caixeiro1, caixeiro2, m, dist, tradutor_volume, tradutor_massa, custo_caminhao)
            if Calculate_FO(corrente1, dist) < Calculate_FO(best, dist):
              best = corrente1
              save = 1
      if save == 1:
        escolha = 1
      else:
        escolha = 6

    if escolha == 6:
      break

  return best


  '''
    while escolha == 2:
      save = 0
      for k in range(0, m):
        caixeiro = k
        corrente1 = Node_exchange(corrente, caixeiro, dist, m, custo_caminhao)
        if Calculate_FO(corrente1[m]) < Calculate_FO(best[m]):
          best = corrente1
          save = 1
      if save == 1:
        escolha = 1
      else:
        escolha = 3

    while escolha == 3:
      save = 0
      for k in range(0, m):
        caixeiro = k
        corrente1 = Opt_3(corrente, caixeiro, dist, m, custo_caminhao)
        if Calculate_FO(corrente1[m]) < Calculate_FO(best[m]):
          best = corrente1
          save = 1
      if save == 1:
        escolha = 1
      else:
        escolha = 4



    while escolha == 1:
    save = 0
    for k in range(0, m-1):
      caixeiro1 = k
      caixeiro2 = k+1
      for i in range(len(corrente[caixeiro1])):
        for j in range(len(corrente[caixeiro2])):
          valor1 = corrente[caixeiro1][i]
          valor2 = corrente[caixeiro2][j]
          corrente1 = swap_inter(corrente, valor1, valor2, caixeiro1, caixeiro2, m, dist, tradutor_volume, tradutor_massa, custo_caminhao)
          if Calculate_FO(corrente1[m]) < Calculate_FO(best[m]):
            best = corrente1
            save = 1
    if save == 1:
      escolha = 1
    else:
      escolha = 2

  while escolha == 2:
    save = 0
    for j in range(0, m):
      caixeiro = j
      for i in range(len(corrente[caixeiro])-1):
        valor1 = corrente[caixeiro][i]
        valor2 = corrente[caixeiro][i+1]
        corrente1 = swap_intra(corrente, corrente[caixeiro][corrente[caixeiro].index(valor1)], corrente[caixeiro][corrente[caixeiro].index(valor2)], caixeiro, dist, m, custo_caminhao)
        if Calculate_FO(corrente1[m]) < Calculate_FO(best[m]):
            best = corrente1
            save = 1
    if save == 1:
      escolha = 2
    else:
      escolha = 3

  while escolha == 3:
    save = 0
    for k in range(0, m-1):
      caixeiro1 = k
      caixeiro2 = k+1
      for i in range(len(corrente[caixeiro1])):
        for j in range(len(corrente[caixeiro2])):
          valor1 = corrente[caixeiro1][i]
          valor2 = corrente[caixeiro2][j]
          corrente1 = shift_inter(corrente, valor1, valor2, caixeiro1, caixeiro2, dist, m, tradutor_volume, tradutor_massa, custo_caminhao) 
        if Calculate_FO(corrente1[m]) < Calculate_FO(best[m]):
            best = corrente1
            save = 1
    if save == 1:
      escolha = 3
    else:
      escolha = 4
  '''

# Simulated Annealing

In [ ]:
def simulated_annealing(estrutura_total, n, m, dist, tradutor_volume, tradutor_massa, custo_caminhao):
  T_min = 10**(-2) ## > 10^5
  T0 = 100000
  SA_max = 4000 ## [100, 2000]
  alpha = 0.9 ## [0.8, 0.9999]
  iter = 0
  T = T0

  melhor_sol = estrutura_total
  corrente = estrutura_total
  while T > T_min:
    while iter < SA_max:
      iter += 1
      corrente1 = LS_VND(corrente, dist, m, n, tradutor_volume, tradutor_massa, custo_caminhao)
      delta = Calculate_FO(corrente1, dist) - Calculate_FO(corrente, dist)
      if delta <= 0:
        corrente = corrente1
        if Calculate_FO(corrente, dist) <= Calculate_FO(melhor_sol, dist):
          sol_otm = corrente
      else:
        x = random.random()
        if x < np.e**(-delta/T):
          corrente = corrente1
    T = T*alpha
  return sol_otm

# GRASP

In [ ]:
def Greedy_construtive(alpha, estrutura_aux, dist, m, tradutor_volume, tradutor_massa, custo_caminhao):  
  lista = [i for i in range(1, n)]
  no_entrante = 0
  while len(lista) > 0:
    tmp = min(estrutura_aux[m])
    caixeiro = estrutura_aux[m].index(tmp)
    if len(estrutura_aux[caixeiro]) == 0:
      no_entrante = 0
    else:
      no_entrante = estrutura_aux[caixeiro][-1]
    minimo = min(dist[no_entrante])
    maximo = max(dist[no_entrante])
    rcl = [i for i in lista if dist[no_entrante][i] <= minimo + alpha*(maximo - minimo)]
    volume = 1000000000
    massa = 100000000
    j = 0
    while estrutura_aux[m+1][caixeiro][1] + volume >= estrutura_aux[m+1][caixeiro][0] and estrutura_aux[m+2][caixeiro][1] + massa >= estrutura_aux[m+2][caixeiro][0]:
      if len(rcl) == 0 or j == len(rcl):
        random_num = random.choice(lista)
        volume = tradutor_volume[random_num]
        massa = tradutor_massa[random_num]
      else:
        random_num = random.choice(rcl)
        volume = tradutor_volume[random_num]
        massa = tradutor_massa[random_num]
        j = j + 1
    estrutura_aux[caixeiro].append(random_num)
    lista.remove(random_num)
    estrutura_aux[m+2][caixeiro][1] = estrutura_aux[m+2][caixeiro][1] + massa
    estrutura_aux[m+1][caixeiro][1] = estrutura_aux[m+1][caixeiro][1] + volume
    estrutura_aux[m][caixeiro] = Calculo_da_FO_Caixeiro(estrutura_aux[caixeiro], dist, custo_caminhao[caixeiro])
  return estrutura_aux

def GRASP(estrutura_total, n, m, dist, tradutor_volume, tradutor_massa, qnt_volume, qnt_massa, custo_caminhao):
  max_iter = 5000
  alpha = 0.45
  for i in range(max_iter):
    estrutura_aux = def_estrutura_de_dados(n, m, qnt_volume, qnt_massa)
    estrutura_aux = Greedy_construtive(alpha, estrutura_aux, dist, m, tradutor_volume, tradutor_massa, custo_caminhao)
    estrutura_aux = LS_VND(estrutura_aux, dist, m, n, tradutor_volume, tradutor_massa, custo_caminhao)
    if Calculate_FO(estrutura_aux, dist) < Calculate_FO(estrutura_total, dist):
      estrutura_total = estrutura_aux
  return estrutura_total

# VNS

In [ ]:
def VNS(estrutura_total, n, m, dist, r, tradutor_volume, tradutor_massa, qnt_volume, qnt_massa, custo_caminhao):
  corrente = estrutura_total
  i = 0
  while i <= r:
    corrente = def_estrutura_de_dados(n, m, qnt_volume, qnt_massa)
    corrente = Greedy_construtive(0.3, corrente, dist, m, tradutor_volume, tradutor_massa, custo_caminhao)
    corrente = LS_VND(corrente, dist, m, n, tradutor_volume, tradutor_massa, custo_caminhao)
    if Calculate_FO(corrente, dist) < Calculate_FO(estrutura_total, dist):
      estrutura_total = corrente
      i = 1
    else:
      i += 1
  return estrutura_total

In [ ]:
cam = 'C:/Users/davij/OneDrive/Documentos/Davi Juliano/Faculdadezinha/PPG-PO/Meta-Heurísticas - Chaves/Instâncias'
chdir(cam)
arqs = []

for c in listdir():
  if isfile(c):
    arqs.append(c)

writer = pd.ExcelWriter('test.xlsx', engine='openpyxl')
for instancia in arqs:
  f = open(instancia,"r")

  ################################ Informações dos Pedidos #######################################
  dataframe_pedidos = pd.read_excel(instancia, sheet_name='Pedido')
  destinos_pedidos = dataframe_pedidos['Destinatário'].tolist()
  destinos_pedidos = list(set(destinos_pedidos))

  ################################ Informações do Destinatário #######################################
  dataframe_locais = pd.read_excel(instancia, sheet_name='Destinatário')
  nome_destinos = dataframe_locais['Código'].tolist()
  decodificador_destinos = {destinos_pedidos[i]:i for i in range(len(destinos_pedidos))}
  destinos_peso_tot = {i:0 for i in range(len(nome_destinos))}
  destinos_volume_tot = {i:0 for i in range(len(nome_destinos))}
  latitude = dataframe_locais['Latitude'].tolist()
  longitude = dataframe_locais['Longitude'].tolist()

  ################################ Informações dos Produtos #######################################
  dataframe_produtos = pd.read_excel(instancia, sheet_name='Produto')
  peso_produtos = dataframe_produtos['Peso (kg)'].tolist()
  volume_produtos = dataframe_produtos['Volume (m³)'].tolist()
  nome_produtos = dataframe_produtos['Código'].tolist()
  decodificador_produtos = {nome_produtos[i]:i for i in range(len(nome_produtos))}

  ################################ Informações dos Veículos #######################################
  dataframe_caminhoes = pd.read_excel(instancia, sheet_name='Veículos')
  volume_list = dataframe_caminhoes['Cubagem (m³)'].tolist()
  peso_list = dataframe_caminhoes['Lotação (kg)'].tolist()
  custo_caminhao = dataframe_caminhoes['Custo / Km'].tolist()

  lista_nos = []
  for i in range(len(latitude)):
    aux_1 = []
    aux_1.append(int(latitude[i]))
    aux_1.append(int(longitude[i]))
    lista_nos.append(aux_1)

  matriz_de_distancias = Euc_2D(lista_nos)
  n = len(matriz_de_distancias)
  m = len(volume_list)

  estrutura_total = def_estrutura_de_dados(n, m, volume_list, peso_list)

  for i in range(len(destinos_pedidos)):
    df_aux = dataframe_pedidos[dataframe_pedidos['Destinatário'] == destinos_pedidos[i]]
    qnt = df_aux['Quantidade'].tolist()
    produto = df_aux['Produto'].tolist()
    peso = 0
    volume = 0
    for j in range(len(produto)):
      indice = decodificador_produtos[produto[j]]
      peso = peso + int(qnt[j]) * float(peso_produtos[indice])
      volume = volume + int(qnt[j]) * float(volume_produtos[indice])
    indice_pedido = decodificador_destinos[destinos_pedidos[i]]
    destinos_peso_tot[indice_pedido] = peso
    destinos_volume_tot[indice_pedido] = volume


FileNotFoundError: ignored

In [ ]:
################################ Informações dos Pedidos #######################################
dataframe_pedidos = pd.read_excel(instancia, sheet_name='Pedido')
destinos_pedidos = dataframe_pedidos['Destinatário'].tolist()
destinos_pedidos = list(set(destinos_pedidos))

################################ Informações do Destinatário #######################################
dataframe_locais = pd.read_excel(instancia, sheet_name='Destinatário')
nome_destinos = dataframe_locais['Código'].tolist()
decodificador_destinos = {destinos_pedidos[i]:i for i in range(len(destinos_pedidos))}
destinos_peso_tot = {i:0 for i in range(len(nome_destinos))}
destinos_volume_tot = {i:0 for i in range(len(nome_destinos))}
latitude = dataframe_locais['Latitude'].tolist()
longitude = dataframe_locais['Longitude'].tolist()

################################ Informações da Origem #######################################
dataframe_produtos = pd.read_excel(arq, sheet_name='Origem')
nome_destinos = dataframe_locais['Código'].tolist()
origem_lista = [decodificador_destinos[nomes] for nomes in nome_destinos]

################################ Informações dos Produtos #######################################
dataframe_produtos = pd.read_excel(instancia, sheet_name='Produto')
peso_produtos = dataframe_produtos['Peso (kg)'].tolist()
volume_produtos = dataframe_produtos['Volume (m³)'].tolist()
nome_produtos = dataframe_produtos['Código'].tolist()
decodificador_produtos = {nome_produtos[i]:i for i in range(len(nome_produtos))}
tempo_inicial_produtos = dataframe_produtos['*início recebimento'].tolist()
tempo_fim_produtos = dataframe_produtos['*Fim recebimento'].tolist()
delta_tempo_produtos = [ tempo_fim_produtos[i] - tempo_inicial_produtos[i] for i in range(len(tempo_inicial_produtos)) ]

################################ Informações dos Veículos #######################################
dataframe_caminhoes = pd.read_excel(instancia, sheet_name='Veículos')
volume_list = dataframe_caminhoes['Cubagem (m³)'].tolist()
peso_list = dataframe_caminhoes['Lotação (kg)'].tolist()
custo_caminhao = dataframe_caminhoes['Custo / Km'].tolist()
origem_caminhao = dataframe_caminhoes['Origem'].tolist()
origem_caminhao = [decodificador_destinos[nomes] for nomes in origem_caminhao]


lista_nos = []
for i in range(len(latitude)):
  aux_1 = []
  aux_1.append(int(latitude[i]))
  aux_1.append(int(longitude[i]))
  lista_nos.append(aux_1)

matriz_de_distancias = Euc_2D(lista_nos)
n = len(matriz_de_distancias)
m = len(volume_list)

estrutura_total = def_estrutura_de_dados(n, m, volume_list, peso_list, delta_tempo_produtos)

for i in range(len(destinos_pedidos)):
  df_aux = dataframe_pedidos[dataframe_pedidos['Destinatário'] == destinos_pedidos[i]]
  qnt = df_aux['Quantidade'].tolist()
  produto = df_aux['Produto'].tolist()
  peso = 0
  volume = 0
  for j in range(len(produto)):
    indice = decodificador_produtos[produto[j]]
    peso = peso + int(qnt[j]) * float(peso_produtos[indice])
    volume = volume + int(qnt[j]) * float(volume_produtos[indice])
  indice_pedido = decodificador_destinos[destinos_pedidos[i]]
  destinos_peso_tot[indice_pedido] = peso
  destinos_volume_tot[indice_pedido] = volume

sa = []
vns = []
grasp = []
tmp_sa = []
tmp_vns = []
tmp_grasp = []

for i in range(2):
  start = time.time()
  estrutura_total = def_estrutura_de_dados(n, m, volume_list, peso_list)
  estrutura_total = Construtive_Insertion(estrutura_total, n, m, matriz_de_distancias, destinos_volume_tot, destinos_peso_tot, custo_caminhao)
  estrutura_total = simulated_annealing(estrutura_total, n, m, matriz_de_distancias, destinos_volume_tot, destinos_peso_tot, custo_caminhao)
  print(estrutura_total)
  end = time.time()
  tmp_sa.append(end-start)
  sa.append(Calculate_FO(estrutura_total, matriz_de_distancias))
  print(Calculate_FO(estrutura_total, matriz_de_distancias))

for i in range(2):
  start = time.time()
  estrutura_total = def_estrutura_de_dados(n, m, volume_list, peso_list)
  estrutura_total = Construtive_Insertion(estrutura_total, n, m, matriz_de_distancias, destinos_volume_tot, destinos_peso_tot, custo_caminhao)
  estrutura_total = VNS(estrutura_total, n, m, matriz_de_distancias, 5000, destinos_volume_tot, destinos_peso_tot, volume_list, peso_list, custo_caminhao)
  print(estrutura_total)
  end = time.time()
  vns.append(Calculate_FO(estrutura_total, matriz_de_distancias))
  tmp_vns.append(end-start)
  print(Calculate_FO(estrutura_total, matriz_de_distancias))

for i in range(2):
  start = time.time()
  estrutura_total = def_estrutura_de_dados(n, m, volume_list, peso_list)
  estrutura_total = Construtive_Insertion(estrutura_total, n, m, matriz_de_distancias, destinos_volume_tot, destinos_peso_tot, custo_caminhao)
  estrutura_total = GRASP(estrutura_total, n, m, matriz_de_distancias, destinos_volume_tot, destinos_peso_tot, volume_list, peso_list, custo_caminhao)
  print(estrutura_total)
  end = time.time()
  tmp_grasp.append(end-start)
  grasp.append(Calculate_FO(estrutura_total, matriz_de_distancias))
  print(Calculate_FO(estrutura_total, matriz_de_distancias))

 
df = pd.DataFrame({'Simulated Annealing':sa, 'Tempo (SA)':tmp_sa, 'GRASP': grasp, 'Tempo (GRASP)':tmp_grasp,  'VNS':vns, 'Tempo (VNS)':tmp_vns})
df.to_excel(writer, sheet_name='Instância1')
writer.save()

[[], [13, 11, 19, 1], [6, 15, 2, 3, 17, 20, 7, 9, 12, 14, 8], [10, 5, 18, 4, 16], [0, 141.1438766733754, 397.9234522799784, 176.11367442066984], [[385.0, 0], [160.16, 6.350822999998908], [398.75, 15.82985900000996], [160.16, 10.708812999998363]], [[33000, 0], [6000, 9205.0], [40000, 16569.0], [6000, 7230.0]]]
715.1810033740237
[[], [7, 9, 12, 20], [8, 14, 1, 11, 19, 13, 5, 2, 15, 6, 4, 18], [17, 3, 16, 10], [0, 152.09685194556195, 452.56871025880037, 166.89146410854994], [[385.0, 0], [160.16, 5.161597999999833], [398.75, 19.082219000018547], [160.16, 8.645677999995858]], [[33000, 0], [6000, 3729.0], [40000, 21098.0], [6000, 8177.0]]]
771.5570263129123
[[], [20, 17, 16], [5, 10, 19, 1, 11, 7, 13, 9, 8, 14, 12], [3, 2, 15, 6, 4, 18], [0, 94.28746304333305, 361.77457434255774, 130.82542926116963], [[385.0, 0], [160.16, 8.111168000000003], [398.75, 15.32766200000001], [160.16, 9.450664999999997]], [[33000, 0], [6000, 8124.0], [40000, 15278.0], [6000, 9602.0]]]
586.8874666470604
[[], [20, 1

In [ ]:
################################ Informações do Destinatário #######################################
dataframe_locais = pd.read_excel('DadosParaDesafio-OtimizaçãoTransporte.xlsx', sheet_name='Destinatário')
nome_destinos = dataframe_locais['Código'].tolist()
decodificador_destinos = {nome_destinos[i]:i for i in range(len(nome_destinos))}
destinos_peso_tot = {i:0 for i in range(len(nome_destinos))}
destinos_volume_tot = {i:0 for i in range(len(nome_destinos))}
latitude = dataframe_locais['Latitude'].tolist()
longitude = dataframe_locais['Longitude'].tolist()

arq = 'new_1DadosParaDesafio-OtimizaçãoTransporte.xlsx'

################################ Informações da Origem #######################################
dataframe_produtos = pd.read_excel(arq, sheet_name='Origem')
nome_destinos = dataframe_locais['Código'].tolist()
origem_lista = [decodificador_destinos[nomes] for nomes in nome_destinos]

################################ Informações dos Produtos #######################################
dataframe_produtos = pd.read_excel(arq, sheet_name='Destinatarios')
peso_produtos = dataframe_produtos['Peso'].tolist()
volume_produtos = dataframe_produtos['Volume'].tolist()


################################ Informações dos Veículos #######################################
dataframe_caminhoes = pd.read_excel(arq, sheet_name='Veiculos')
volume_list = dataframe_caminhoes['Veiculo_Volume'].tolist()
peso_list = dataframe_caminhoes['Veiculo_Peso'].tolist()
custo_caminhao = dataframe_caminhoes['Veiculo_custo'].tolist()
origem_caminhao = dataframe_caminhoes['Origem'].tolist()
origem_caminhao = [decodificador_destinos[nomes] for nomes in origem_caminhao]

lista_nos = []
for i in range(len(latitude)):
  aux_1 = []
  aux_1.append(int(latitude[i]))
  aux_1.append(int(longitude[i]))
  lista_nos.append(aux_1)

matriz_de_distancias = Euc_2D(lista_nos)
n = len(volume_produtos)+1
m = len(volume_list)

estrutura_total = def_estrutura_de_dados(n, m, volume_list, peso_list)

for i in range(len(peso_produtos)):
  peso = peso_produtos[i]
  volume = volume_produtos[i]
  destinos_peso_tot[i] = peso
  destinos_volume_tot[i] = volume

print(destinos_peso_tot)
print(destinos_volume_tot)
print(estrutura_total)

estrutura_total1 = def_estrutura_de_dados(n, m, volume_list, peso_list, tempo_list)
print(estrutura_total1)

{0: 1870, 1: 1220, 2: 2536, 3: 1335, 4: 735, 5: 1322, 6: 2036, 7: 864, 8: 2301, 9: 372, 10: 1132, 11: 1086, 12: 2304, 13: 342, 14: 2690, 15: 3039, 16: 1161, 17: 672, 18: 1126, 19: 2379, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0

In [ ]:


print(matriz_de_distancias[0][13])
print(matriz_de_distancias[13][1])
print(matriz_de_distancias[1][11])
print(matriz_de_distancias[11][19])
print(matriz_de_distancias[19][0])
print()

10.295630140987
21.02379604162864
23.259406699226016
11.0
17.204650534085253



In [ ]:
print(custo_caminhao)
custo_total = 0
for i, n in enumerate(custo_caminhao):
  print('------------------')
  print(estrutura_total[i])
  if estrutura_total[i]:
    print(i, n)
    custo_rota_caminhao = Calculo_da_FO_Caixeiro(estrutura_total[i], matriz_de_distancias, n)
    print(custo_rota_caminhao)
    custo_total = custo_total + custo_rota_caminhao
print(custo_total)

[4.5, 3.45, 4.8, 3.45]
------------------
[]
------------------
[2, 4, 3, 5, 20]
1 3.45
248.50298307663815
------------------
[13, 8, 18, 1, 15, 6, 17, 16, 19, 7, 11, 10]
2 4.8
718.1942569672304
------------------
[12, 9, 14]
3 3.45
216.68476080846614
1183.3820008523346
